# Working with CO Aggregated Data

Pre-processing input data for a smoother upload experience of the state data to the WaDE 2.0 database.
Using geopandas to read in shp file, and coverting to WKT for ReportingUnit geometry.

Separating tasks out into two processes. 1) create aggregate amount time series input.  2) create reporting unit input.

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
import geopandas as gpd # the library that lets us read in shapefiles
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [3]:
#Setting work directory, reading inputs, creating dataframe
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Colorado/AggregatedAmounts"
os.chdir(workingDir)

#CSV input file
fileInput = "RawInputData/DWR__Surface_Water_Supply_Index_Component_by_HUC_input.xlsx"
df = pd.read_excel(fileInput)

#Shapefile input
shapeInput = gpd.read_file('C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Colorado/AggregatedAmounts/RawInputData/HUC8/CO_HUC8.shp')
dfshape = pd.DataFrame(shapeInput)

In [4]:
#check csv input
df.head(3)

,Basin,HUC8,HUC8 Name,Report Year,Report Month,TimeframeStart,TimeframeEnd,Component Type,Component ID,Component Name,Component Volume,Component NEP
0,Rio Grande,13010002,Alamosa-Trinchera,2010,10,10/01/2010,10/31/2010,ReservoirStorage,8008160,MOUNTAIN HOME,1484.0,15.60
1,Rio Grande,13010002,Alamosa-Trinchera,2010,10,10/01/2010,10/31/2010,ReservoirStorage,MTNRESCO,TERRACE RESERVOIR,1085.0,23.90
2,Rio Grande,13010002,Alamosa-Trinchera,2010,11,11/01/2010,11/30/2010,ReservoirStorage,8008160,MOUNTAIN HOME,1322.0,15.52


In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df.dtypes)

Basin                object
HUC8                  int64
HUC8 Name            object
Report Year           int64
Report Month          int64
TimeframeStart       object
TimeframeEnd         object
Component Type       object
Component ID         object
Component Name       object
Component Volume    float64
Component NEP       float64
dtype: object


In [6]:
#check shp input
dfshape.head(3)

,TNMID,METASOURCE,SOURCEDATA,SOURCEORIG,SOURCEFEAT,LOADDATE,GNIS_ID,AREAACRES,AREASQKM,STATES,HUC8,NAME,Shape_Leng,Shape_Le_1,Shape_Area,geometry
0,{4CCAA733-584D-4347-A7F3-4E664ADA8B9B},None,None,None,None,2012-06-11,0,1104144.63,4468.32,"CO,NM",11080001,Canadian Headwaters,3.943636,3.943636,0.450856,"POLYGON ((-104.18034 36.92065, -104.18082 36.9..."
1,{A5C1EEA1-80EE-49EC-9473-DF41AD004E7F},None,None,None,None,2017-09-20,0,2082907.01,8429.23,"CO,NM",13020101,Upper Rio Grande,6.441450,6.441450,0.847277,"POLYGON ((-105.23391 37.03586, -105.23421 37.0..."
2,{72C226B6-F4BB-484B-8ADD-88C896D0DF72},None,None,None,None,2012-06-11,0,2021138.98,8179.27,"CO,NM",13020102,Rio Chama,4.926198,4.926198,0.822519,"POLYGON ((-106.58821 37.13834, -106.58788 37.1..."


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfshape.dtypes)

TNMID           object
METASOURCE      object
SOURCEDATA      object
SOURCEORIG      object
SOURCEFEAT      object
LOADDATE        object
GNIS_ID          int64
AREAACRES      float64
AREASQKM       float64
STATES          object
HUC8            object
NAME            object
Shape_Leng     float64
Shape_Le_1     float64
Shape_Area     float64
geometry      geometry
dtype: object


In [8]:
#Dropping fields we don't need.
df = df.drop(['Basin', 'Component NEP', 'Report Month'], axis=1)
df

,HUC8,HUC8 Name,Report Year,TimeframeStart,TimeframeEnd,Component Type,Component ID,Component Name,Component Volume
0,13010002,Alamosa-Trinchera,2010,10/01/2010,10/31/2010,ReservoirStorage,8008160,MOUNTAIN HOME,1484.0
1,13010002,Alamosa-Trinchera,2010,10/01/2010,10/31/2010,ReservoirStorage,MTNRESCO,TERRACE RESERVOIR,1085.0
2,13010002,Alamosa-Trinchera,2010,11/01/2010,11/30/2010,ReservoirStorage,8008160,MOUNTAIN HOME,1322.0
3,13010002,Alamosa-Trinchera,2010,11/01/2010,11/30/2010,ReservoirStorage,MTNRESCO,TERRACE RESERVOIR,1316.0
4,13010002,Alamosa-Trinchera,2010,12/01/2010,12/31/2010,ReservoirStorage,8008160,MOUNTAIN HOME,1829.0
...,...,...,...,...,...,...,...,...,...
15383,14050001,Upper Yampa,2020,9/01/2020,9/30/2020,PrevMoStreamflow,9239500,YAMPA RIVER AT STEAMBOAT SPRINGS,3798.0
15384,14050001,Upper Yampa,2020,9/01/2020,9/30/2020,PrevMoStreamflow,9242500,"ELK RIVER NEAR MILNER, CO",2887.0
15385,14050001,Upper Yampa,2020,9/01/2020,9/30/2020,PrevMoStreamflow,9246200,ELKHEAD CREEK ABOVE LONG GULCH,82.0
15386,14050001,Upper Yampa,2020,9/01/2020,9/30/2020,ReservoirStorage,9237495,STAGECOACH RESERVOIR NR OAK CREEK,34700.0


In [9]:
#Changing datatype of used date fields. 
df['TimeframeStart'] = pd.to_datetime(df['TimeframeStart'], errors = 'coerce')
df['TimeframeStart'] = pd.to_datetime(df["TimeframeStart"].dt.strftime('%m/%d/%Y'))

df['TimeframeEnd'] = pd.to_datetime(df['TimeframeEnd'], errors = 'coerce')
df['TimeframeEnd'] = pd.to_datetime(df["TimeframeEnd"].dt.strftime('%m/%d/%Y'))

In [10]:
WSTypeDict = {
    "MOUNTAIN HOME" : "Lake",
    "TERRACE RESERVOIR" : "Reservoir",
    "ALAMOSA CREEK ABOVE TERRACE RESERVOIR" : "Reservoir",
    "TRINCHERA CK" : "Creek",
    "SANGRE DE CRISTO" : "River",
    "UTE CREEK" : "Creek",
    "CULEBRA CREEK AT SAN LUIS" : "Creek",
    "LEMON RESERVOIR" : "Reservoir",
    "ANIMAS RIVER AT DURANGO" : "River",
    "FLORIDA RIVER INFLOW TO LEMON RESERVOIR" : "River",
    "FLORIDA RIVER BELOW LEMON RESERVOIR NR DURANGO" : "River",
    "CLEAR CREEK RESERVOIR" : "Reservoir",
    "TURQUOISE LAKE" : "Lake",
    "TWIN LAKES RESERVOIR" : "Lake",
    "HOMESTAKE RESERVOIR" : "Reservoir",
    "ARKANSAS RIVER AT SALIDA" : "River",
    "BOYD LAKE" : "Lake",
    "CARTER LAKE" : "Lake",
    "LAKE LOVELAND RESERVOIR" : "Reservoir",
    "LONE TREE RESERVOIR" : "Reservoir",
    "MARIANO RESERVOIR" : "Reservoir",
    "LAKE GRANBY" : "Lake",
    "WILLOW CREEK RESERVOIR" : "Reservoir",
    "BIG THOMPSON R AT MOUTH, NR DRAKE, CO" : "River",
    "GREEN MOUNTAIN RESERVOIR" : "Reservoir",
    "BLUE RIVER INFLOW TO GREEN MOUNTAIN RES" : "Reservoir",
    "BLACK HOLLOW RESERVOIR" : "Reservoir",
    "CACHE LA POUDRE" : "Lake",
    "CHAMBERS LAKE" : "Lake",
    "COBB LAKE" : "Lake",
    "FOSSIL CREEK RESERVOIR" : "Reservoir",
    "HALLIGAN RESERVOIR" : "Reservoir",
    "HORSETOOTH RESERVOIR" : "Reservoir",
    "WINDSOR RESERVOIR" : "Reservoir",
    "CACHE LA POUDRE R AT CANYON MOUTH" : "River",
    "CACHE LA POUDRE R AT CANYON MOUTH NEAR FORT COLLINS" : "River",
    "CLEAR CREEK AT GOLDEN" : "Creek",
    "WILLIAMS FORK RESERVOIR" : "Reservoir",
    "WOLFORD MOUNTAIN RESERVOIR" : "Reservoir",
    "COLORADO RIVER NEAR DOTSERO" : "River",
    "VEGA RESERVOIR" : "Reservoir",
    "COLORADO RIVER NEAR CAMEO" : "River",
    "PLATORO RESERVOIR" : "Reservoir",
    "CONEJOS RIVER NEAR MOGOTE" : "River",
    "EAGLE RIVER BELOW GYPSUM" : "River",
    "TAYLOR PARK RESERVOIR" : "Reservoir",
    "TAYLOR R INF TO TAYLOR PARK RESERVOIR" : "Reservoir",
    "EAST RIVER AT ALMONT" : "River",
    "TAYLOR RIVER BELOW TAYLOR PARK RESERVOIR" : "Reservoir",
    "CUCHARAS RESERVOIR" : "Reservoir",
    "HUERFANO RIVER NEAR REDWING" : "River",
    "CUCHARAS RIVER AT BOYD RANCH NR LA VETA" : "River",
    "LITTLE SNAKE RIVER NEAR LILY" : "River",
    "GUNNISON RIVER NR GRAND JUNCTION" : "River",
    "YAMPA RIVER NEAR MAYBELL" : "River",
    "JACKSON GULCH RESERVOIR" : "Reservoir",
    "MANCOS RIVER NEAR MANCOS" : "River",
    "LONG HOLLOW RESERVOIR" : "Reservoir",
    "LA PLATA RIVER AT HESPERUS" : "River",
    "BARR LAKE" : "Lake",
    "MILTON RESERVOIR" : "Reservoir",
    "STANDLEY RESERVOIR" : "Reservoir",
    "HORSECREEK RESERVOIR" : "Reservoir",
    "SOUTH PLATTE RIVER AT SOUTH PLATTE" : "River",
    "BEAR CREEK ABV EVERGREEN" : "Creek",
    "SAINT VRAIN CREEK AT LYONS" : "Creek",
    "BOULDER CREEK NEAR ORODELL" : "Creek",
    "SOUTH BOULDER CK NR ELDORADO SPRINGS, CO" : "Creek",
    "EMPIRE RESERVOIR" : "Reservoir",
    "JACKSON LAKE RESERVOIR" : "Reservoir",
    "JULESBURG RESERVOIR" : "Reservoir",
    "POINT OF ROCKS RESERVOIR" : "Reservoir",
    "PREWITT RESERVOIR" : "Reservoir",
    "RIVERSIDE RESERVOIR" : "Reservoir",
    "PAONIA RESERVOIR" : "Reservoir",
    "NORTH FORK GUNNISON R NR SOMERSET" : "River",
    "NORTH PLATTE R NR NORTHGATE" : "River",
    "PIEDRA RIVER NEAR ARBOLES" : "River",
    "TRINIDAD LAKE" : "Lake",
    "PURGATOIRE RIVER AT TRINIDAD" : "River",
    "RIO GRANDE RESERVOIR" : "Reservoir",
    "SANTA MARIA RESERVOIR" : "Reservoir",
    "CONTINENTAL RESERVOIR" : "Reservoir",
    "RIO GRANDE NEAR DEL NORTE" : "River",
    "RUEDI RESERVOIR" : "Reservoir",
    "ROARING FORK AT GLENWOOD SPRINGS" : "Spring",
    "SAGUACHE CREEK NEAR SAGUACHE, CO" : "Creek",
    "SAN MIGUEL RIVER NEAR PLACERVILLE" : "River",
    "ANTERO RESERVOIR" : "Reservoir",
    "ELEVENMILE CANYON RESERVOIR" : "Reservoir",
    "SPINNEY MOUNTAIN RESERVOIR" : "Reservoir",
    "ELEVENMILE CANYON RESV INFLOW" : "Reservoir",
    "ELEVENMILE CANYON RE NEAR LAKE GEORGE" : "Lake",
    "GROSS RESERVOIR" : "Reservoir",
    "MARSHALL RESERVOIR" : "Reservoir",
    "BUTTONROCK (RALPH PRICE) RESERVOIR" : "Reservoir",
    "TERRY RESERVOIR" : "Reservoir",
    "UNION RESERVOIR" : "Reservoir",
    "VOUGA RESERVOIR NEAR DOYLEVILLE" : "Reservoir",
    "TOMICHI CREEK AT GUNNISON, CO" : "Creek",
    "RIDGEWAY RESERVOIR" : "Reservoir",
    "UNCOMPAHGRE RIVER AT COLONA" : "River",
    "PUEBLO RESERVOIR" : "Reservoir",
    "PUEBLO RESERVOIR INFLOW" : "Reservoir",
    "ARKANSAS RIVER ABOVE PUEBLO" : "River",
    "ADOBE CREEK RESERVOIR" : "Reservoir",
    "JOHN MARTIN RESERVOIR" : "Reservoir",
    "MEREDITH RESERVOIR" : "Reservoir",
    "LAKE HENRY" : "Lake",
    "GROUNDHOG RESERVOIR" : "Reservoir",
    "MCPHEE RESERVOIR" : "Reservoir",
    "DOLORES RIVER BELOW MCPHEE RESERVOIR" : "Reservoir",
    "BLUE MESA RESERVOIR" : "Reservoir",
    "MORROW POINT RESERVOIR" : "Reservoir",
    "FRUITLAND RESERVOIR" : "Reservoir",
    "CRAWFORD RESERVOIR" : "Reservoir",
    "SILVER JACK RESERVOIR" : "Reservoir",
    "LAKE FORK AT GATEVIEW, CO" : "Lake",
    "GUNNISON R INF TO BLUE MESA RESERVOIR" : "Reservoir",
    "GUNNISON RIVER NEAR GUNNISON, CO" : "River",
    "VALLECITO RESERVOIR" : "Reservoir",
    "SAN JUAN RIVER NEAR CARRACAS" : "River",
    "LOS PINOS RIVER NEAR BAYFIELD" : "River",
    "CHEESMAN LAKE" : "Lake",
    "DILLON RESERVOIR" : "Reservoir",
    "WHITE RIVER NEAR MEEKER" : "River",
    "STAGECOACH RESERVOIR NR OAK CREEK" : "Reservoir",
    "YAMCOLO RESERVOIR" : "Reservoir",
    "YAMPA RIVER AT STEAMBOAT SPRINGS" : "River",
    "ELK RIVER NEAR MILNER, CO" : "River",
    "ELKHEAD CREEK ABOVE LONG GULCH" : "Creek",
    "ELKHEAD RIVER NEAR MILNER" : "River"}


def assignWSType(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        return np.nan
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = WSTypeDict[String1]
        except:
            outList = "Unspecified"
    return outList

df['WaterSourceTypeCV'] = df.apply(lambda row: assignWSType(row['Component Name']), axis=1)

In [11]:
#Final Data Type Check
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df.dtypes)

HUC8                          int64
HUC8 Name                    object
Report Year                   int64
TimeframeStart       datetime64[ns]
TimeframeEnd         datetime64[ns]
Component Type               object
Component ID                 object
Component Name               object
Component Volume            float64
WaterSourceTypeCV            object
dtype: object


In [12]:
df

,HUC8,HUC8 Name,Report Year,TimeframeStart,TimeframeEnd,Component Type,Component ID,Component Name,Component Volume,WaterSourceTypeCV
0,13010002,Alamosa-Trinchera,2010,2010-10-01,2010-10-31,ReservoirStorage,8008160,MOUNTAIN HOME,1484.0,Lake
1,13010002,Alamosa-Trinchera,2010,2010-10-01,2010-10-31,ReservoirStorage,MTNRESCO,TERRACE RESERVOIR,1085.0,Reservoir
2,13010002,Alamosa-Trinchera,2010,2010-11-01,2010-11-30,ReservoirStorage,8008160,MOUNTAIN HOME,1322.0,Lake
3,13010002,Alamosa-Trinchera,2010,2010-11-01,2010-11-30,ReservoirStorage,MTNRESCO,TERRACE RESERVOIR,1316.0,Reservoir
4,13010002,Alamosa-Trinchera,2010,2010-12-01,2010-12-31,ReservoirStorage,8008160,MOUNTAIN HOME,1829.0,Lake
...,...,...,...,...,...,...,...,...,...,...
15383,14050001,Upper Yampa,2020,2020-09-01,2020-09-30,PrevMoStreamflow,9239500,YAMPA RIVER AT STEAMBOAT SPRINGS,3798.0,River
15384,14050001,Upper Yampa,2020,2020-09-01,2020-09-30,PrevMoStreamflow,9242500,"ELK RIVER NEAR MILNER, CO",2887.0,River
15385,14050001,Upper Yampa,2020,2020-09-01,2020-09-30,PrevMoStreamflow,9246200,ELKHEAD CREEK ABOVE LONG GULCH,82.0,Creek
15386,14050001,Upper Yampa,2020,2020-09-01,2020-09-30,ReservoirStorage,9237495,STAGECOACH RESERVOIR NR OAK CREEK,34700.0,Reservoir


In [13]:
#The Output - csv file
df.to_csv('RawInputData/P_COagg.csv', index=False)

### Creting Geometry Input File for ReportingUnits

Geometry field takes up too much space, and is mostly repeated and redundant.  Can improve by making shorter, separate reporting unit input file.

In [18]:
dfru = df
dfru = dfru.drop_duplicates(subset=['HUC8 Name']).reset_index()
dfru.head(3)

,index,HUC8,HUC8 Name,Report Year,TimeframeStart,TimeframeEnd,Component Type,Component ID,Component Name,Component Volume,WaterSourceTypeCV
0,0,13010002,Alamosa-Trinchera,2010,2010-10-01,2010-10-31,ReservoirStorage,8008160,MOUNTAIN HOME,1484.0,Lake
1,682,14080104,Animas,2010,2010-10-01,2010-10-31,ReservoirStorage,9009070,LEMON RESERVOIR,12062.0,Reservoir
2,980,11020001,Arkansas Headwaters,2010,2010-10-01,2010-10-31,ReservoirStorage,7007020,CLEAR CREEK RESERVOIR,6500.0,Reservoir


In [19]:
#Dropping fields we don't need.
dfru = dfru.drop(['index'], axis=1)
dfru.head(3)

,HUC8,HUC8 Name,Report Year,TimeframeStart,TimeframeEnd,Component Type,Component ID,Component Name,Component Volume,WaterSourceTypeCV
0,13010002,Alamosa-Trinchera,2010,2010-10-01,2010-10-31,ReservoirStorage,8008160,MOUNTAIN HOME,1484.0,Lake
1,14080104,Animas,2010,2010-10-01,2010-10-31,ReservoirStorage,9009070,LEMON RESERVOIR,12062.0,Reservoir
2,11020001,Arkansas Headwaters,2010,2010-10-01,2010-10-31,ReservoirStorage,7007020,CLEAR CREEK RESERVOIR,6500.0,Reservoir


In [20]:
#Fixing datatype to propery match dfs.
dfshape['HUC8'] = dfshape['HUC8'].astype(int)

#transfer WKT gemetry from dfshape
def retrieveGeometry(colrowValue, dfshape):
    if (colrowValue == '') or (pd.isnull(colrowValue)):
        outList = ''
    else:
        ml = dfshape.loc[(dfshape['HUC8'] == colrowValue), 'geometry']
        if not(ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

dfru['Geometry'] = dfru.apply(lambda row: retrieveGeometry(row['HUC8'], dfshape), axis=1)
dfru.head(3)

,HUC8,HUC8 Name,Report Year,TimeframeStart,TimeframeEnd,Component Type,Component ID,Component Name,Component Volume,WaterSourceTypeCV,Geometry
0,13010002,Alamosa-Trinchera,2010,2010-10-01,2010-10-31,ReservoirStorage,8008160,MOUNTAIN HOME,1484.0,Lake,POLYGON ((-105.4855449057728 37.57782530799165...
1,14080104,Animas,2010,2010-10-01,2010-10-31,ReservoirStorage,9009070,LEMON RESERVOIR,12062.0,Reservoir,POLYGON ((-107.5162357963449 37.83438947439475...
2,11020001,Arkansas Headwaters,2010,2010-10-01,2010-10-31,ReservoirStorage,7007020,CLEAR CREEK RESERVOIR,6500.0,Reservoir,POLYGON ((-106.2142136427454 39.38309409231857...


In [21]:
#The Output - csv file
dfru.to_csv('RawInputData/P_COagg_ru.csv', index=False)